In [5]:
import json
import time
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
from pathlib import Path


# Getting File

In [5]:
# Opening JSON file
start = time.time()
with open('dict_spottings.json') as json_file:
    data = dict(json.load(json_file))

In [8]:
def removeType(string):
    return string[:-4]

In [9]:
videoLocation = 'D:/BOBSL/BOBSL200G/bobsl/videos'
videoTitles = os.listdir(videoLocation)

videoName = videoTitles[0]
targetVideo = videoLocation + '/' +videoName

videoTitlesEdited = list(map(removeType, videoTitles))

# Only Loading in files that I have Downloaded

In [10]:
wordsTrain = list(dict(data['train']).keys())

train = []
for word in wordsTrain:
    wordData = data['train'][word]
    for i in range(len(wordData['names'])):
        fileName = wordData['names'][i]
        if fileName in videoTitlesEdited:
            globalTime = wordData['global_times'][i]
            probs = wordData['probs'][i]
            train.append((word, fileName, globalTime, probs))

In [11]:
wordsVal = list(dict(data['val']).keys())

val = []
for word in wordsVal:
    wordData = data['val'][word]
    for i in range(len(wordData['names'])):
        fileName = wordData['names'][i]
        if fileName in videoTitlesEdited:
            globalTime = wordData['global_times'][i]
            probs = wordData['probs'][i]
            val.append((word, fileName, globalTime, probs))

In [12]:
wordsTest = list(dict(data['test']).keys())

test = []
for word in wordsTest:
    wordData = data['test'][word]
    for i in range(len(wordData['names'])):
        fileName = wordData['names'][i]
        if fileName in videoTitlesEdited:
            globalTime = wordData['global_times'][i]
            probs = wordData['probs'][i]
            test.append((word, fileName, globalTime, probs))

In [13]:
print(len(train))
print(len(val))
print(len(test))

2758672
112184
414790


## Create new file showing information about files i have

In [ ]:
f1 = open("train.txt", "w")
for line in train:
    f1.write(str(line) + '\n')
f1.close()

f2 = open("val.txt", "w")
for line in val:
    f2.write(str(line) + '\n')
f2.close()

f3 = open("test.txt", "w")
for line in test:
    f3.write(str(line) + '\n')
f3.close()

# Creating smaller videos

In [14]:
sortedVideoList = []
for videoName in videoTitlesEdited:
    targeVideoInfo = []
    for wordInfo in val: # wordInfo = (word, fileName, globalTime, probs)
        if wordInfo[1] == videoName:
            targeVideoInfo.append(wordInfo)
    if len(targeVideoInfo) > 0:  
        sortedVideoList.append(targeVideoInfo)
sortedVideoList

[[('1', '5213407827313563421', 63.2, 0.79419),
  ('1', '5213407827313563421', 188.8, 0.82758),
  ('1', '5213407827313563421', 457.76, 0.8011),
  ('1', '5213407827313563421', 457.76, 0.8011),
  ('1', '5213407827313563421', 477.76, 0.72512),
  ('1', '5213407827313563421', 516.32, 0.71467),
  ('1', '5213407827313563421', 799.52, 0.72857),
  ('1', '5213407827313563421', 1051.04, 0.73824),
  ('1', '5213407827313563421', 1123.52, 0.74813),
  ('1', '5213407827313563421', 1351.68, 0.74029),
  ('1', '5213407827313563421', 1351.68, 0.74029),
  ('1', '5213407827313563421', 1423.68, 0.74991),
  ('1', '5213407827313563421', 1423.68, 0.74991),
  ('1', '5213407827313563421', 1437.76, 0.71505),
  ('1', '5213407827313563421', 1453.76, 0.72748),
  ('100', '5213407827313563421', 1049.76, 0.75347),
  ('1st', '5213407827313563421', 215.52, 0.7307),
  ('1st', '5213407827313563421', 765.28, 0.70436),
  ('2', '5213407827313563421', 162.56, 0.81283),
  ('2', '5213407827313563421', 187.84, 0.86731),
  ('2', '52

In [18]:
video1 = sortedVideoList[0]
FPS = 25
parts = []
for info in video1:
    globalTime = info[2]
    frameStart = globalTime
    frameEnd = globalTime + 1.5
    parts.append((frameStart, frameEnd, info[0]))
parts = sorted(parts, key=lambda x: x[0])
f4 = open("check2.txt", "w")
for line in parts:
    f4.write(str(line) + '\n')
f4.close()

In [20]:

videoLocation = 'D:/BOBSL/BOBSL200G/bobsl/videos'
videoTitles = os.listdir(videoLocation)

saveLocation = 'D:/Thesis/Video-Segments-BOBSL'

videoName = video1[0][1]

In [15]:
def extract_segment(input_file, start_time, end_time, output_file):
    cap = cv2.VideoCapture(input_file)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calculate start and end frame
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)

    # Set video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Extract frames and write to output file
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if frame_num < start_frame:
            continue
        if frame_num > end_frame:
            break

        out.write(frame)

    # Release resources
    cap.release()
    out.release()

In [25]:
videoName = video1[0][1]
targetVideo = videoLocation + '/' + videoName + '.mp4'
saveLocation = 'D:/Thesis/Video-Segments-BOBSL'
j = 0
for start, end, word in parts:
    if j == 100:
        break
    
    output  = saveLocation + '/' + videoName + '-' + word + '-'+ str(int(start)) +'-' + str(int(end)) + '.mp4'
    path = Path(output)
    i = 1
    print(path)
    # while(path.is_file()):
    #     print("b")
    #     continue
    print(output, start, end, word)
    extract_segment(targetVideo, start, end, output)
    j += 1


D:\Thesis\Video-Segments-BOBSL\5213407827313563421-to-0-1.mp4
D:/Thesis/Video-Segments-BOBSL/5213407827313563421-to-0-1.mp4 0.16 1.66 to
D:\Thesis\Video-Segments-BOBSL\5213407827313563421-bcc-3-5.mp4
D:/Thesis/Video-Segments-BOBSL/5213407827313563421-bcc-3-5.mp4 3.68 5.18 bcc
D:\Thesis\Video-Segments-BOBSL\5213407827313563421-south-africa-4-5.mp4
D:/Thesis/Video-Segments-BOBSL/5213407827313563421-south-africa-4-5.mp4 4.16 5.66 south-africa
D:\Thesis\Video-Segments-BOBSL\5213407827313563421-blow-4-6.mp4
D:/Thesis/Video-Segments-BOBSL/5213407827313563421-blow-4-6.mp4 4.8 6.3 blow
D:\Thesis\Video-Segments-BOBSL\5213407827313563421-special-5-7.mp4
D:/Thesis/Video-Segments-BOBSL/5213407827313563421-special-5-7.mp4 5.6 7.1 special
D:\Thesis\Video-Segments-BOBSL\5213407827313563421-all-5-7.mp4
D:/Thesis/Video-Segments-BOBSL/5213407827313563421-all-5-7.mp4 5.92 7.42 all
D:\Thesis\Video-Segments-BOBSL\5213407827313563421-got-6-7.mp4
D:/Thesis/Video-Segments-BOBSL/5213407827313563421-got-6-7.mp4

# ELAR Dataset

In [3]:
from pympi import Eaf

### Function to extract signs from a video

In [2]:
def extract_segment_adpated(input_file, timesWord, output_loc, videoName):
    cap = cv2.VideoCapture(input_file)
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Extract frames and write to output file
    frames = {}
    print('video: start frame')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        frames[frame_num] = frame

    print('video: end frames')
    # Release resources
    cap.release()


    i = 0
    print('Start: creating segments')
    for start, end, word in timesWord:
        # print(i, start, end, word)
        output_file = output_loc + word +'_'+ videoName +'_' + str(int(start)) + '_' + str(int(end)) + '.mp4'

        # Calculate start and end frame
        start_frame = int(start * fps * 0.001)
        end_frame = int(end * fps * 0.001)

        # Set video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
        for frame_num in range(start_frame, end_frame):
            try:
                frame = frames[frame_num]
            except KeyError as error:
                continue
            out.write(frame)
        out.release()
        i += 1
    print('Finished: creating segments')

### Create segments by sign for one video file

In [ ]:
videoName = 'AAM1A1c2a'
videoLocation = 'D:/Auslan/AUSLANRECO-I0569/data/ELAR Data/AAM1A1c2a - The Boy Who Cried Wolf'
targetVideo = videoLocation + '/' + videoName + '.mp4'
saveLocation = 'D:/Thesis/Video-Segments-AUSLAN'

fileEAF = 'D:/Auslan/AUSLANRECO-I0569/data/ELAR Data/AAM1A1c2a - The Boy Who Cried Wolf/AAM1A1c2a.eaf'
eaf = Eaf(fileEAF)

j = 0
timeWords = []
for key in list(eaf.tiers['RH-IDgloss'][0].keys()):
    if j == 250:
        break
    
    start = eaf.timeslots[eaf.tiers['RH-IDgloss'][0][key][0]]
    end = eaf.timeslots[eaf.tiers['RH-IDgloss'][0][key][1]]
    word = eaf.tiers['RH-IDgloss'][0][key][2]
    timeWords.append((start, end, word))
print('Words times finished')

extract_segment_adpated(targetVideo, timeWords, saveLocation + '/', videoName)


### Convert All video Files

In [12]:
folderLocation = 'D:/Auslan/AUSLANRECO-I0569/data/ELAR Data/'
folderList = os.listdir(folderLocation)
saveLocationParent = 'D:/Thesis/Video-Segments-AUSLAN/' 

i = 0
for folderName in folderList:
    if folderName == '! Annotated Files':
        continue
    # Video File 
    videoLocation = folderLocation + folderName
    videoName = folderName.split()[0]
    targetVideo = videoLocation + '/' + videoName + '.mp4'
    # ELAR File
    fileEAF = videoLocation + '/' + videoName + '.eaf'
    try:
        eaf = Eaf(fileEAF)
    except FileNotFoundError as error:
        continue
    # Save Location
    path = os.path.join(saveLocationParent, videoName)
    try:
        os.mkdir(path)
    except OSError as error:
        print(error)

    saveLocation = saveLocationParent + videoName
    print(saveLocation)
    timeWords = []
    for key in list(eaf.tiers['RH-IDgloss'][0].keys()):
        start = eaf.timeslots[eaf.tiers['RH-IDgloss'][0][key][0]]
        end = eaf.timeslots[eaf.tiers['RH-IDgloss'][0][key][1]]
        word = eaf.tiers['RH-IDgloss'][0][key][2]
        timeWords.append((start, end, word))
    print('Words times finished')
    extract_segment_adpated(targetVideo, timeWords, saveLocation + '/', videoName)
    i+=1

D:/Thesis/Video-Segments-AUSLAN/AAM1A1c2a
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM1A2c6ii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM1A2c6iv
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM2A1c2b
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM2A2c6ii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM2A2c6iv
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAPB1c2b
Words times finished
video: start fra

C:\Users\KAI\AppData\Roaming\Python\Python39\site-packages\pympi\Elan.py:1471: UserWarning: Parsing unknown version of ELAN spec... This could result in errors...
  warnings.warn('Parsing unknown version of ELAN spec... '


video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW1B2c6iii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW1B2c7a
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW2A1c2b
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW2A2c6ii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW2A2c7a
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/ARGB2c6iii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thes

### Test Stuff

In [85]:
output  = saveLocation + '/' + 'test' + '-' + word + '-'+ str(int(start)) +'-' + str(int(end)) + '.mp4'
extract_segment(targetVideo, 100, 110, output)

### How many include both Subtitles

In [19]:
folderLocation = 'D:/Auslan/AUSLANRECO-I0569/data/ELAR Data/'
folderList = os.listdir(folderLocation)
i=0
j=0
for folderName in folderList:
        j+=1
        if folderName == '! Annotated Files':
                continue
        videoName = folderName.split()[0]
        videoLocation = folderLocation + folderName
        # print(folderLocation + folderName)
        # ELAR File
        fileEAF = videoLocation + '/' + videoName + '.eaf'
        # print(fileEAF)
        try:
                eaf = Eaf(fileEAF)
        except FileNotFoundError as error:
                continue
        if len(eaf.tiers['FreeTransl'][0]):
                i += 1
print(i)
print(j)


C:\Users\KAI\AppData\Roaming\Python\Python39\site-packages\pympi\Elan.py:1471: UserWarning: Parsing unknown version of ELAN spec... This could result in errors...
  warnings.warn('Parsing unknown version of ELAN spec... '


103
211


## Create Pose Data

In [4]:
import os
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd

## Old Implementaion of mediapipe
### DO NOT USE

In [45]:
# initialize Pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


videoLocation = 'D:/Thesis/Video-Auslan-sorted'

for word in os.listdir(videoLocation):
    
    wordLocation = videoLocation + '/' + word
    print(wordLocation)
    # Loop through every video
    for videoName in os.listdir(wordLocation): 
        coordinates = []
        
        print('opening: ' + wordLocation + '/' + videoName)
        vid = cv2.VideoCapture(wordLocation + '/' + videoName)

        if (vid.isOpened()== False): 
            print("Error opening video stream or file")
        frames = 0
        with mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose:
            while vid.isOpened():
                success, image = vid.read()
                frames += 1
                if not success:
                    print("Ignoring empty camera frame.")
                    break
                image_height, image_width, _ = image.shape

                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = pose.process(image)
                if not results.pose_landmarks:
                    continue
                else:
                    coordinates.append(results.pose_landmarks.landmark)

                # Draw the hand annotations on the image.
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # process image
                results = pose.process(image)
                # results

                # draw landmarks
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                # Display image
                # cv2.imshow('MediaPipe Pose', image)
                if cv2.waitKey(5) & 0xFF == 27:
                    break
        vid.release()
        cv2.destroyAllWindows()
        
        coor = []
        for frame in coordinates:
            coorFrame = []
            for landmarks in frame:
                coorFrame.append((landmarks.x * -1, landmarks.y * -1, landmarks.z * -1))
            coor.append(coorFrame)
        coor = np.array(coor)
        if len(coor) > 0:
            print("saving: ", videoName[:-4])
            np.savetxt("D:/Thesis/ELAR-Data/arrayData/"+ videoName[:-4] +".npy", coor.reshape(coor.shape[0], -1))
            np.savetxt("D:/Thesis/ELAR-Data/arrayData/"+ videoName[:-4] +"_shape.npy", np.array(coor.shape))
            coorLoad = np.loadtxt("D:/Thesis/test.npy")
            coorLoad = coorLoad.reshape(coorLoad.shape[0], coorLoad.shape[1] // coor.shape[2], coor.shape[2])
    break
        


D:/Thesis/Video-Auslan-sorted/ABOUT1
opening: D:/Thesis/Video-Auslan-sorted/ABOUT1/ABOUT1_BFSA1c2a_1630_1990.mp4
Ignoring empty camera frame.
saving:  ABOUT1_BFSA1c2a_1630_1990
opening: D:/Thesis/Video-Auslan-sorted/ABOUT1/ABOUT1_BRCA1c2a_27365_27645.mp4
Ignoring empty camera frame.
opening: D:/Thesis/Video-Auslan-sorted/ABOUT1/ABOUT1_BRCA1c2a_29505_29985.mp4
Ignoring empty camera frame.
opening: D:/Thesis/Video-Auslan-sorted/ABOUT1/ABOUT1_SBS1A3c7a_950_1280.mp4
Ignoring empty camera frame.
saving:  ABOUT1_SBS1A3c7a_950_1280


## New Implementaion of mediapipe

In [9]:
# initialize Pose estimator
model_path = './mediapipe/pose_landmarker_heavy.task'
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)

videoLocation = 'D:/Thesis/Video-Auslan-sorted'
i = 0
upToWord = False
for word in os.listdir(videoLocation):
    i = 1
    wordLocation = videoLocation + '/' + word
    print(wordLocation)
    # Loop through every video
    if word != 'DEER' and not upToWord:
        continue
    else:
        upToWord = True
    for videoName in os.listdir(wordLocation): 
        coordinates = []
        frames = 0
        print('opening: ' + wordLocation + '/' + videoName)
        vid = cv2.VideoCapture(wordLocation + '/' + videoName)
        with PoseLandmarker.create_from_options(options) as landmarker:
            while vid.isOpened():
                success, image = vid.read()
                frames += 1
                if not success:
                    break
                mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
                pose_landmarker_result = landmarker.detect_for_video(mp_image, frames)
                if len(pose_landmarker_result.pose_landmarks) > 0:
                    coordinates.append(pose_landmarker_result.pose_landmarks)
        vid.release()
        coor = []
        for frame in coordinates:
            coorFrame = []
            frame = frame[0]
            for landmarks in frame:
                coorFrame.append((landmarks.x * -1, landmarks.y * -1, landmarks.z * -1))
            coor.append(coorFrame)
        coor = np.array(coor)
        if len(coor) > 0:
            print("saving: ", videoName[:-4])
            np.savetxt("D:/Thesis/ELAR-Data/arrayData/"+ videoName[:-4] +".npy", coor.reshape(coor.shape[0], -1))
            np.savetxt("D:/Thesis/ELAR-Data/arrayData/"+ videoName[:-4] +"_shape.npy", np.array(coor.shape))
            coorLoad = np.loadtxt("D:/Thesis/test.npy")
            coorLoad = coorLoad.reshape(coorLoad.shape[0], coorLoad.shape[1] // coor.shape[2], coor.shape[2])

D:/Thesis/Video-Auslan-sorted/ABOUT1
D:/Thesis/Video-Auslan-sorted/ABOUT2
D:/Thesis/Video-Auslan-sorted/ABOUT2(7)
D:/Thesis/Video-Auslan-sorted/ABOUT2(B)
D:/Thesis/Video-Auslan-sorted/ABOUT2(FALSE-START)
D:/Thesis/Video-Auslan-sorted/ACCEPT
D:/Thesis/Video-Auslan-sorted/ACCOMMODATION-2H
D:/Thesis/Video-Auslan-sorted/ACCURATE
D:/Thesis/Video-Auslan-sorted/ACHIEVE
D:/Thesis/Video-Auslan-sorted/ACTIVITY-THERE
D:/Thesis/Video-Auslan-sorted/ADULT
D:/Thesis/Video-Auslan-sorted/AFRAID
D:/Thesis/Video-Auslan-sorted/AFTERNOON
D:/Thesis/Video-Auslan-sorted/AFTERNOON3
D:/Thesis/Video-Auslan-sorted/AFTERNOON4
D:/Thesis/Video-Auslan-sorted/AGAIN
D:/Thesis/Video-Auslan-sorted/AGAIN(8)
D:/Thesis/Video-Auslan-sorted/AGAIN-2H
D:/Thesis/Video-Auslan-sorted/AGAIN-AND-AGAIN
D:/Thesis/Video-Auslan-sorted/AGES
D:/Thesis/Video-Auslan-sorted/AGREE
D:/Thesis/Video-Auslan-sorted/AIM
D:/Thesis/Video-Auslan-sorted/ALIVE
D:/Thesis/Video-Auslan-sorted/ALL
D:/Thesis/Video-Auslan-sorted/ALL-2H
D:/Thesis/Video-Auslan-

In [49]:
newCoor = coor

In [46]:
oldCoor = coor

In [ ]:
frame = 0
print(max(oldCoor[frame].T[0]), max(newCoor[frame].T[0]))

-0.34257861971855164 -0.35253024101257324


In [ ]:
labelLocation = 'D:/Thesis/ELAR-Data/label.csv'
labelDf = pd.read_csv(labelLocation)
labelDict = dict(np.array(labelDf))


In [33]:
Y = []
X = []
coorDataLocation = "D:/Thesis/ELAR-Data/arrayData/"
dirData = os.listdir(coorDataLocation)
for i, data in enumerate(dirData):
    if 'shape' in data:
        continue
    fileLocation = coorDataLocation + data
    shapeLocation = coorDataLocation + dirData[i + 1]
    coorLoad = np.loadtxt(fileLocation)
    coorShape= np.loadtxt(shapeLocation)
    coorLoad = coorLoad.reshape(coorLoad.shape[0], coorLoad.shape[1] // int(coorShape[2]), int(coorShape[2]))
    X.append(coorLoad)
    Y.append(labelDict[data.split('_')[0]])

IndexError: tuple index out of range